# **1. Importación de *modules***

In [137]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# **2. Importación del dataset *properatti.csv***

In [138]:
data = pd.read_csv("../data/properatti.csv", sep = ",", low_memory=False) 
#data.head(3)
data.shape

(121220, 26)

# **3. Tratamiento de los datos**

## **3.1. Selección del subdataset AMBA**

In [139]:
#dejamos solo lo de AMBA
data = data[(data["state_name"] == 'Capital Federal') | (data["state_name"] == 'Bs.As. G.B.A. Zona Sur') | (data["state_name"] == 'Bs.As. G.B.A. Zona Norte') | (data["state_name"] == 'Bs.As. G.B.A. Zona Oeste')]
print(data.shape)

(81150, 26)


## **3.2. Limpieza e imputaciones**

### 3.2.1. Dropeo de columnas no informativas

In [140]:
cols2keep = ['property_type', 'state_name', 'place_name','place_with_parent_names','price_aprox_usd', 'surface_total_in_m2','surface_covered_in_m2','price_usd_per_m2', 'rooms', 'description', 'title', 'properati_url']
data = data.loc[:, cols2keep]
#print(df_sub)

### 3.2.2. Chequeo de valores nulos

In [141]:
#Chequeo valores nulos
cant_nulos_por_campo = data.apply(lambda x: x.isnull().sum(), axis=0)
percent_nulos_por_campo = data.apply(lambda x: (100 * x.isnull().sum() / data.shape[0]).round(2), axis=0)
summary_nulos_por_campo = pd.DataFrame({ 'cant': cant_nulos_por_campo, 'percent': percent_nulos_por_campo ,'tipo': data.dtypes})
print(summary_nulos_por_campo)

                          cant  percent     tipo
property_type                0     0.00   object
state_name                   0     0.00   object
place_name                  23     0.03   object
place_with_parent_names      0     0.00   object
price_aprox_usd           8656    10.67  float64
surface_total_in_m2      22792    28.09  float64
surface_covered_in_m2     8505    10.48  float64
price_usd_per_m2         29515    36.37  float64
rooms                    51211    63.11  float64
description                  1     0.00   object
title                        0     0.00   object
properati_url                0     0.00   object


### 3.2.3. Imputación de la columna *rooms*


In [142]:
#Comprobamos el tipo y los valores.
print("tipo  : ", data.rooms.dtype)
print("nulos : ", data.rooms.isnull().sum())
data.rooms.unique()

tipo  :  float64
nulos :  51211


array([nan,  1.,  4.,  3.,  2.,  6.,  5., 10.,  7., 17., 22.,  8.,  9.,
       11., 13., 12., 25., 14., 16., 18., 15., 32.])

In [143]:
totalVentas = data.state_name.notnull().sum()
totalSinCuartos = data.rooms.isnull().sum()
print('Total Registros \t\t\t', data.state_name.notnull().sum() )
print('Total sin ambientes \t\t\t',data.rooms.isnull().sum() )
print('Porcentaje sin ambientes \t\t', round( (totalVentas- totalSinCuartos) / totalVentas * 100, 2), ' %' )

Total Registros 			 81150
Total sin ambientes 			 51211
Porcentaje sin ambientes 		 36.89  %


In [144]:
data['title'] = data.title.str.upper()
data['description'] = data.description.str.upper()

In [145]:
reemplazo_dic = {"UNO":"1", "DOS":"2", "TRES":"3", "CUATRO":"4", "CINCO":"5", "SEIS":"6", "SIETE":"7", "OCHO":"8",
             "NUEVE":"9", "DIEZ":"10", "MONOAMBIENTE":"1 AMBIENTE", "MONOAMB" : "1 AMBIENTE", "UN":"1", "AMBIENTE DIVISIBLE":"1 AMBIENTE",
             "MONO AMBIENTE": "1 AMBIENTE","DORMITORIOS": "AMBIENTE","DORMITORIO": "AMBIENTE","HABITACIONES": "AMBIENTE","HABITACION": "AMBIENTE"}
for key in reemplazo_dic.keys():
    data.description = data.description.str.replace(key, reemplazo_dic[key], regex=False)

for key in reemplazo_dic.keys():
    data.title = data.title.str.replace(key, reemplazo_dic[key], regex=False)

In [146]:
regularexpArray=["(\d+)AMB","(\d+) AMB"]

for regularexp in regularexpArray:
    controlRooms = data[(data.rooms.isnull()) ]
    controlRooms.rooms = controlRooms.title.str.extract(regularexp).astype(float)
    data.update(controlRooms)
    controlRooms = data[(data.rooms.isnull()) ]
    controlRooms.rooms = controlRooms.description.str.extract(regularexp).astype(float) 
    data.update(controlRooms)
    
# se agrega un ambiente mas
regularexpArray=["(\d+)DORM","(\d+) DORM","(\d+)HABITACIO","(\d+) HABITACIO"]

for regularexp in regularexpArray:
    controlRooms = data[(data.rooms.isnull()) ]
    controlRooms.rooms = controlRooms.title.str.extract(regularexp).astype(float) + 1
    data.update(controlRooms)
    controlRooms = data[(data.rooms.isnull()) ]
    controlRooms.rooms = controlRooms.description.str.extract(regularexp).astype(float) + 1
    data.update(controlRooms)    
controlRooms = data[(data.rooms.isnull())]

C:\Users\msaied\Anaconda3\envs\ds\lib\site-packages\pandas\core\generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [147]:
totalVentas = data.state_name.notnull().sum()
totalSinCuartos = data.rooms.isnull().sum()
porcentajeSinCuartos = round( (totalVentas- totalSinCuartos) / totalVentas * 100, 2)
print('Total Registros \t\t\t', totalVentas )
print('Total sin Ambientes \t\t\t', totalSinCuartos )
print('Porcentaje con ambientes \t\t', porcentajeSinCuartos, ' %' )

Total Registros 			 81150
Total sin Ambientes 			 10685
Porcentaje con ambientes 		 86.83  %


In [ ]:
# Pasar todas las columnas a minusculas

data_lower = data.applymap(lambda x: x if np.isreal(x) else str(x).lower())
# comparo los tipos de datos antes y después de pasar a minúsculas:
#print(data_lower.dtypes == data.dtypes)
#print(data_lower.head(3))

### 3.2.4. Dropeo de duplicados

In [1]:
#elimino duplicados 
data.drop_duplicates(keep="first")

NameError: name 'data' is not defined

### 3.2.5. Limpieza de *outliers* en *price_usd_per_m2*

In [150]:
#búsqueda y reemplazo de outliers (de más de 2 std, 95%) por NaN en las columnas numéricas, en un solo paso
df_sub = data.loc[:, 'price_usd_per_m2']
lim = np.abs((df_sub - df_sub.mean()) / df_sub.std(ddof=0)) < 2
data.loc[:, 'price_usd_per_m2'] = df_sub.where(lim, np.nan)
#data.head(3)
data.shape
#data.dropna()

(81150, 12)

### 3.2.6. Dropeo de *NaNs*

In [151]:
data.dropna(axis=0, how='any', subset=['property_type', 'state_name', 'place_name', 'price_aprox_usd', 'surface_total_in_m2','surface_covered_in_m2', 'rooms', 'price_usd_per_m2'], inplace=True)
data.shape

In [153]:
# Último chequeo de que no hay nans en las columnas de trabajo (feature y targets)

cant_nulos_por_campo = data.apply(lambda x: x.isnull().sum(), axis=0)
percent_nulos_por_campo = data.apply(lambda x: (100 * x.isnull().sum() / data.shape[0]).round(2), axis=0)
summary_nulos_por_campo = pd.DataFrame({ 'cant': cant_nulos_por_campo, 'percent': percent_nulos_por_campo ,'tipo': data.dtypes})
print(summary_nulos_por_campo)

                         cant  percent     tipo
property_type               0      0.0   object
state_name                  0      0.0   object
place_name                  0      0.0   object
place_with_parent_names     0      0.0   object
price_aprox_usd             0      0.0  float64
surface_total_in_m2         0      0.0  float64
surface_covered_in_m2       0      0.0  float64
price_usd_per_m2            0      0.0  float64
rooms                       0      0.0  float64
description                 0      0.0   object
title                       0      0.0   object
properati_url               0      0.0   object


## **3.3. Creacion de columnas *dummies***

### 3.3.1. *Amenities*

In [154]:
#fracciono la columna properti_url para sacar la nube de palabras mas repetidas
import re
patron_url = re.compile(pattern="_", flags =re.IGNORECASE)
lista_url = data["properati_url"].apply(lambda x : patron_url.split(x))
serie_palabras = pd.Series(np.hstack(lista_url))
#serie_palabras.value_counts().head(20).plot(kind="bar")

In [155]:
serie_palabras.value_counts().head(50)

venta                       42662
departamento                27323
garage                      21795
lavadero                    19749
balcon                      17206
parrilla                    16518
piscina                     14135
luminoso                    14032
suite                       13137
casa                        12777
toilette                    11959
placard                     10993
terraza                     10784
vestidor                     8882
jardin                       7775
patio                        6598
dependencias                 6451
sum                          5961
aire-acondicionado           5700
baulera                      5494
amenities                    5423
gimnasio                     4763
estrenar                     4394
lujoso                       4275
vista                        3850
subte-linea-d                3639
quincho                      3599
hidromasaje                  3170
subte-linea-b                2986
subte-linea-a 

In [156]:
# a partir de la nube de palabras selecciono las que son buenos adicionales
adicionales = ["garage", "balcon", "parrilla", "piscina", "terraza", "patio", "jardin", "quincho", "sum", "amenities", "baulera", "gimnasio", "subte-linea-d", "subte-linea-b", "subte-linea-a", "subte-linea-h", "subte-linea-e" ]


In [157]:
#elimino el primer elemento de lista_url para no tener el elemento con el http: etc
for sublist in lista_url:
  del sublist[0]

In [158]:
lista_url

0         [venta, ph, mataderos, lavadero, patio, inmobi...
2         [venta, departamentos, mataderos, lavadero, pl...
6         [venta, ph, munro, lavadero, patio, garage, al...
7         [venta, departamentos, belgrano, lavadero, pis...
8         [venta, departamentos, belgrano, lavadero, pis...
                                ...                        
121154    [venta, casa, la-plata, suite, parrilla, pisci...
121158    [venta, departamento, recoleta, pueyrredon-av-...
121215    [venta, departamento, belgrano, balcon, suite,...
121216    [venta, casa, beccar, suite, hidromasaje, jard...
121217    [venta, departamento, villa-urquiza, holmberg,...
Name: properati_url, Length: 42717, dtype: object

In [159]:
#creo una función que compare la lista de palabras con la lista de listas
#y me da como resultado una lista de listas de palabras true/false segun coincida o no 
def buscador_palabras(quebuscar, dondebuscar):
  listadeextras = []
  for listas in dondebuscar:
    extras = []
    for palabra in quebuscar:
      if palabra in listas:
        extras.append(True)
      else:
        extras.append(False)
    listadeextras.append(extras)
  #print(listadeextras)
  return listadeextras     

In [160]:
#aplico la funcion a mi lista "adicionales" y "lista_url"
#chequeo que tenga la misma longitud de data
resultado = buscador_palabras(adicionales,lista_url)
len(resultado)

42717

In [161]:
#convierto resultado en dataframe, y renombro las columnas por la lista de palabras adicionales
df = pd.DataFrame(resultado)
df.columns = ["garage", "balcon", "parrilla", "piscina", "terraza", "patio", "jardin", "quincho", "s.u.m.", "amenities", "baulera", "gimnasio","subte-linea-d", "subte-linea-b", "subte-linea-a", "subte-linea-h", "subte-linea-e"]
df

,garage,balcon,parrilla,piscina,terraza,patio,jardin,quincho,s.u.m.,amenities,baulera,gimnasio,subte-linea-d,subte-linea-b,subte-linea-a,subte-linea-h,subte-linea-e
0,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42712,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False
42713,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False
42714,True,True,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False
42715,True,False,True,True,False,False,True,True,False,False,False,False,False,False,False,False,False


In [162]:
#uno el dataframe original con el nuevo generado de true/false
#data = pd.merge(data,df,left_index=True, right_index=True)
data = data.join(df)
data.columns
#antes me puso los dos indices como resultado del merge, y tuve que sacar la primera columna
#data.drop(columns=data.columns[0], axis=1,inplace=True)
#data.columns

Index(['property_type', 'state_name', 'place_name', 'place_with_parent_names',
       'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
       'price_usd_per_m2', 'rooms', 'description', 'title', 'properati_url',
       'garage', 'balcon', 'parrilla', 'piscina', 'terraza', 'patio', 'jardin',
       'quincho', 's.u.m.', 'amenities', 'baulera', 'gimnasio',
       'subte-linea-d', 'subte-linea-b', 'subte-linea-a', 'subte-linea-h',
       'subte-linea-e'],
      dtype='object')

In [163]:
data.shape

(42717, 29)

### 3.3.2. *State name*

**creé las dummies sin drop_first para poder hacer el join**

In [164]:
df_sn = pd.get_dummies(data["state_name"])

In [165]:
data = data.join(df_sn)
data.columns

Index(['property_type', 'state_name', 'place_name', 'place_with_parent_names',
       'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
       'price_usd_per_m2', 'rooms', 'description', 'title', 'properati_url',
       'garage', 'balcon', 'parrilla', 'piscina', 'terraza', 'patio', 'jardin',
       'quincho', 's.u.m.', 'amenities', 'baulera', 'gimnasio',
       'subte-linea-d', 'subte-linea-b', 'subte-linea-a', 'subte-linea-h',
       'subte-linea-e', 'Bs.As. G.B.A. Zona Norte', 'Bs.As. G.B.A. Zona Oeste',
       'Bs.As. G.B.A. Zona Sur', 'Capital Federal'],
      dtype='object')

In [166]:
data.shape

(42717, 33)

### 3.3.3. *Place name*

**creé las dummies sin drop_first para poder hacer el join**

In [167]:
df_pn = pd.get_dummies(data["place_name"], prefix="pn")

In [168]:
data = data.join(df_pn)
data.columns

Index(['property_type', 'state_name', 'place_name', 'place_with_parent_names',
       'price_aprox_usd', 'surface_total_in_m2', 'surface_covered_in_m2',
       'price_usd_per_m2', 'rooms', 'description',
       ...
       'pn_Villa de Mayo', 'pn_Villa del Parque',
       'pn_Village Golf & Tennis Country Club', 'pn_Virasoro Village',
       'pn_Virrey del Pino', 'pn_Virreyes', 'pn_Wilde', 'pn_William Morris',
       'pn_Zelaya', 'pn_coordenadas 34.255511'],
      dtype='object', length=484)

In [169]:
data.shape

(42717, 484)

### 3.3.4. Dropeo de columnas innecesarias y una de cada *dummy*

In [170]:
#data = data.drop( axis = 1, columns = ['description', 'title', 'properati_url', 'Bs.As. G.B.A. Zona Sur','pn_Virreyes'], inplace= True)
data.shape

# **4. Exportación del dataset resultante a un nuevo *.csv***

In [172]:
# para exportar resultados
data.to_csv(r'../data/properatti_tp2.csv', index = False, header=True)

In [ ]:
# para usar el nuevo dataset desde el archivo
# data = pd.read_csv("../data/properatti_tp2.csv", sep = ",", low_memory=False) 
# data.head(3)
# data.shape